WebScrapper for EWeb courses Canavan

In [1]:
#imports
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import sys
from selenium.webdriver.support.select import Select
import pandas as pd
import re

In [2]:
#Term select input
x = input("Would you like to select the last Semester?: ")
def selTerm():
    if x != 'yes'|'no':
        print('Invalid Entry, please state "yes" or "no')
        selTerm()


In [3]:
#Driver for firefox and loading the Eweb url
driver = webdriver.Firefox()
driver.implicitly_wait(10)

driver.get("https://ssb-prod.ec.easternct.edu/PROD/bwskfcls.p_termsel")

In [4]:
# select term
elem = driver.find_elements(by = 'id', value = 'term_id')
if x == 'yes':
	s = Select(elem[0])
	s.select_by_index(2)

In [5]:
# submit term
elem= driver.find_elements(by = 'tag name', value = 'input')
elem[1].click()

In [6]:
#selecting both
both = driver.find_element(by = 'id', value = 'oc_id')
both.click()

In [7]:
#searching
btn = driver.find_element(by = 'name', value = 'SUB_BTN')
btn.click()

In [8]:
    # Locate all tables on the page
tbl = driver.find_elements("tag name", "table")


In [9]:
#This is where all the information will be stored and appened to for .csv exporting
info = []

for tbl_index, table in enumerate(tbl[2:], start=3):  # Enumerate
    
    # Locate the rows in the respective table
    rows = table.find_elements("tag name", "tr")
    
    # Loop through all rows starting with the 4th row 
    for row_index, row in enumerate(rows[3:], start=4):  # Enumerate from 4th row
        
        # Locate all the columns in this row
        col = row.find_elements("tag name", "td")
        
        # Using the columns that hold desired data(crse,sec,day,time,room,instrcutor)
        indices = [3,4, 5, 10, 11, 20, 21]
        data = []
        for col_index in indices:
            # Avoid error if column is not present
            if col_index < len(col):
                data.append(col[col_index].text)
            else:
                data.append("N/A")
        
        # Add the extracted data to info
        info.append(data)

# Creating of Pandas data frame
df = pd.DataFrame(info, columns=["Sub","Crse", "Sec", "Day", "Time", "Room", "Instructor"])

#Resolving blanks
for row in range(len(df)):  # Loop through row indices
    if df.iloc[row, 2] == ' ':  # Check if Sec has ' '
        df.iloc[row, 0] = df.iloc[row-1, 0]  # Copy values from the previous row
        df.iloc[row, 1] = df.iloc[row-1, 1] 
        df.iloc[row, 2] = df.iloc[row-1, 2]

# Export the DataFrame to a CSV file
df.to_csv("AllCourses.csv", index=False)

print("Data exported")


Data exported


In [10]:
#Close the webpage
driver.quit()

In [11]:
# Modify instructor names by removing all instances of '(P)'
df['Instructor'] = df['Instructor'].str.replace(r'\(P\)', "", regex=True).str.strip()

# Filter out rows where the instructor field contains invalid names
df = df[~df['Instructor'].isin(['TBA', '()', '', ' ()', ')', '('])]

# Create a new column that combines Day and Time
# (This is optional if you prefer to work directly with the two columns.)
df['DayTime'] = df.apply(lambda row: f"{row['Day']}({row['Time']})", axis=1)

# Function to group times by day for each instructor
def group_day_times(group):
    # Create a dictionary to hold sets of times for each day
    day_groups = {}
    for _, row in group.iterrows():
        day = row['Day']
        time = row['Time']
        # Add the time into the corresponding day group
        day_groups.setdefault(day, set()).add(time)
    # Format the grouped times into the desired string format
    formatted = []
    for day, times in day_groups.items():
        # Join multiple times for the same day with commas
        times_str = ", ".join(sorted(times))
        formatted.append(f"{day}({times_str})")
    # Optionally sort the formatted segments (for consistent order)
    return ", ".join(sorted(formatted))

# Group by instructor and apply the grouping function
df_grouped = df.groupby('Instructor').apply(group_day_times).reset_index(name='Schedule')

# Export the grouped instructor schedule to a CSV file
df_grouped.to_csv("InstructorSchedules.csv", index=False)

print("Instructor schedule exported successfully.")



Instructor schedule exported successfully.


C:\Users\stevo\AppData\Local\Temp\ipykernel_19764\3974826748.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_grouped = df.groupby('Instructor').apply(group_day_times).reset_index(name='Schedule')


In [12]:

# 1. Read the CSV file that has the Instructor and Schedule columns.
# (Ensure this CSV contains the raw, unaltered schedule strings.)
df = pd.read_csv("InstructorSchedules_Cleaned.csv")

# Remove any space + "()" at the end of the instructor name.
df['Instructor'] = df['Instructor'].str.replace(r"\s*\(\)\s*$", "", regex=True).str.strip()

# We no longer need to convert the Schedule column since it's already a formatted string.
# If your CSV has a 'Time' column that is a string like:
# "['09:30 am-10:45 am']", then you'd use ast.literal_eval.
# But if the column instead contains something like:
# "TR(11:00 am-12:15 pm, 12:30 pm-01:45 pm)",
# then you should leave it as is.

# For demonstration, we assume the column is named 'Schedule'
# and contains the formatted schedule string.

# Group by Instructor, combining unique schedule strings.
# Here we simply join the unique schedule entries with a comma.
df_grouped = (
    df.groupby('Instructor')['Schedule']
      .apply(lambda series: ", ".join(sorted(series.unique())))
      .reset_index()
)

# Sort the grouped DataFrame alphabetically by Instructor.
df_grouped = df_grouped.sort_values(by='Instructor').reset_index(drop=True)

# Save the final result to a new CSV file.
df_grouped.to_csv("InstructorGroupedTimes.csv", index=False)

print("Done! Instructors have been grouped, times combined, and the file is saved.")


Done! Instructors have been grouped, times combined, and the file is saved.


In [13]:
# 1. Read the CSV file that has the Instructor and Schedule columns.
df = pd.read_csv("InstructorGroupedTimes.csv")

# Remove any trailing space plus "()" from the Instructor names.
df['Instructor'] = df['Instructor'].str.replace(r"\s*\(\)\s*$", "", regex=True).str.strip()

# In this case, we assume the Schedule column already contains formatted schedule strings.
# For grouping, we'll combine unique schedule entries.
df_grouped = (
    df.groupby('Instructor')['Schedule']
      .apply(lambda schedules: ", ".join(sorted(set(schedules))))
      .reset_index()
)

# Sort the grouped DataFrame alphabetically by Instructor.
df_grouped = df_grouped.sort_values(by='Instructor').reset_index(drop=True)

# Save the final result to a new CSV file.
df_grouped.to_csv("InstructorFinal.csv", index=False)

print("Done! Instructors have been grouped, times combined, and the file is saved.")

Done! Instructors have been grouped, times combined, and the file is saved.


In [14]:

# 1. Read the cleaned instructor schedule CSV file.
df = pd.read_csv("InstructorClassSchedulesFinal.csv")

# 2. Convert the Schedule column (list-like strings) into a single string per row.
df['Schedule'] = df['Schedule'].astype(str)  # Ensure all values are treated as strings

# 3. Combine Instructor and Schedule into a single string per row.
df['Combined'] = df['Instructor'] + ": " + df['Schedule']

# 4. Save the new file with just the combined column.
df[['Combined']].to_csv("InstructorSchedulesCombined.csv", index=False, header=False)

print("InstructorSchedulesCombined.csv has been created with instructors and schedules combined.")


InstructorSchedulesCombined.csv has been created with instructors and schedules combined.


In [15]:
df = pd.read_csv("InstructorSchedulesCombined.csv", header=None, names=["Combined"])

# 2. Remove all instances of "(TBA)" from the combined string.
df["Combined"] = df["Combined"].str.replace(r"\(TBA\)", "", regex=True).str.strip()

# 3. Save the cleaned file.
df.to_csv("InstructorSchedulesFinalCleaned.csv", index=False, header=False)

print("InstructorSchedulesFinalCleaned.csv has been created with all (TBA) instances removed.")

InstructorSchedulesFinalCleaned.csv has been created with all (TBA) instances removed.


GITHUB, BLANKS, Downloads, GITHUB REPOSITORY
DEVELOP WEB SCRAPER FOR OH

2/21/25
Think about questions you want to answer with prolog. Facts and relationships you need to create!
ADD days to Courses!
Format names to be one string
Format both to have consistent format
Hold off - Seperate Course times out(maybe)
Format GitHub